In [ ]:
!pip install dotenv
!pip install gluonts
!pip install lightning
!pip install jaxtyping
!pip install hydra-core
!pip install --upgrade datasets

In [ ]:
#!git clone https://github.com/GiuliaGhisolfi/TSFM-ZeroShotEval
#%cd TSFM-ZeroShotEval

In [ ]:
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

short_datasets = "solar/10T solar/H solar/D solar/W jena_weather/10T jena_weather/H jena_weather/D " \
"bitbrains_fast_storage/5T bitbrains_fast_storage/H bitbrains_rnd/5T bitbrains_rnd/H bizitobs_application " \
"bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"

med_long_datasets = "solar/10T solar/H jena_weather/10T jena_weather/H " \
"bitbrains_fast_storage/5T bitbrains_rnd/5T bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"

# Get union of short and med_long datasets
all_datasets = list(set(short_datasets.split() + med_long_datasets.split()))

dataset_properties_map = json.load(open("data/dataset_properties.json"))

In [ ]:
from src.load_data import load_gift_data

load_gift_data()

In [ ]:
from gluonts.ev.metrics import (
    MSE,
    MAE,
    MASE,
    MAPE,
    SMAPE,
    MSIS,
    RMSE,
    NRMSE,
    ND,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(
        quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ),
]

In [ ]:
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

model_name = "moirai_moe_small" # "morai_base", "moirai_large", "moirai_moe_base", "moirai_moe_small"
#"Salesforce/moirai-1.1-R-small", "Salesforce/moirai-1.1-R-base", "Salesforce/moirai-1.1-R-large"
#"Salesforce/moirai-moe-1.0-R-base", "Salesforce/moirai-moe-1.0-R-small"

In [ ]:
from gluonts.model import evaluate_model, evaluate_forecasts
import csv
import os
import time
from gluonts.time_feature import get_seasonality
from gift_eval.data_moirai_moe import Dataset

# Iterate over all available datasets

output_dir = "results/moirai"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

pretty_names = {
    "saugeenday": "saugeen",
    "temperature_rain_with_missing": "temperature_rain",
    "kdd_cup_2018_with_missing": "kdd_cup_2018",
    "car_parts_with_missing": "car_parts",
}

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, f"{model_name}_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
        ]
    )

In [ ]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
for ds_name in all_datasets[-1:]:
    ds_key = ds_name.split("/")[0]
    print(f"Processing dataset: {ds_name}")
    terms = ["short", "medium", "long"]
    for term in terms:
        if (
            term == "medium" or term == "long"
        ) and ds_name not in med_long_datasets.split():
            continue

        if "/" in ds_name:
            ds_key = ds_name.split("/")[0]
            ds_freq = ds_name.split("/")[1]
            ds_key = ds_key.lower()
            ds_key = pretty_names.get(ds_key, ds_key)
        else:
            ds_key = ds_name.lower()
            ds_key = pretty_names.get(ds_key, ds_key)
            ds_freq = dataset_properties_map[ds_key]["frequency"]

        ds_config = f"{ds_key}/{ds_freq}/{term}"

        # Initialize the dataset, since Moirai support multivariate time series forecast, it does not require
        # to convert the original data into univariate
        # to_univariate = False if Dataset(name=ds_name, term=term,to_univariate=False).target_dim == 1 else True
        to_univariate = False
        dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)

        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained("Salesforce/moirai-moe-1.0-R-base"),#.to("cuda"),
            prediction_length=dataset.prediction_length,
            context_length=1000,
            patch_size=32,
            num_samples=20, # Reduced from 100 to reduce memory usage
            target_dim=dataset.target_dim,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=dataset.past_feat_dynamic_real_dim,
        )

        season_length = get_seasonality(dataset.freq)

        """
        predictor = model.create_predictor(batch_size=32)

        res = evaluate_model(
        predictor,
        test_data=dataset.test_data,
        metrics=metrics,
        batch_size=512,
        axis=None,
        mask_invalid_label=True,
        allow_nan_forecast=False,
        seasonality=season_length,
        )
        """

        predictor = model.create_predictor(batch_size=32)
        forecasts = predictor.predict(dataset.test_data)

        res = evaluate_forecasts(
            forecasts=forecasts,
            test_data=dataset.test_data,
            metrics=metrics,
            axis=None,
            batch_size=512,
            mask_invalid_label=True,
            allow_nan_forecast=False,
            seasonality=season_length,
        )

        # Append the results to the CSV file
        with open(csv_file_path, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(
                [
                    ds_config,
                    model_name,
                    res["MSE[mean]"][0],
                    res["MSE[0.5]"][0],
                    res["MAE[0.5]"][0],
                    res["MASE[0.5]"][0],
                    res["MAPE[0.5]"][0],
                    res["sMAPE[0.5]"][0],
                    res["MSIS"][0],
                    res["RMSE[mean]"][0],
                    res["NRMSE[mean]"][0],
                    res["ND[0.5]"][0],
                    res["mean_weighted_sum_quantile_loss"][0],
                    dataset_properties_map[ds_key]["domain"],
                    dataset_properties_map[ds_key]["num_variates"],
                ]
            )

        print(f"Results for {ds_name} have been written to {csv_file_path}")

In [ ]:
predictor = model.create_predictor(batch_size=32, device='cpu')
forecasts = predictor.predict(dataset.test_data)

In [ ]:
predictor = model.create_predictor(batch_size=32)
forecasts = predictor.predict(dataset.test_data)

res = evaluate_forecasts(
    forecasts=forecasts,
    test_data=dataset.test_data,
    metrics=metrics,
    axis=None,
    batch_size=512,
    mask_invalid_label=True,
    allow_nan_forecast=False,
    seasonality=season_length,
)